In [11]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder \
        .master("spark://de-taxi.europe-west1-b.c.dynamic-shift-363106.internal:7077") \
        .appName("test") \
        .getOrCreate()

In [2]:
spark

In [4]:
df_green=spark.read.parquet('../data/part/green/*/*')

In [5]:
df_green=df_green.withColumnRenamed('lpep_pickup_datetime', "pickup_datetime") \
        .withColumnRenamed('lpep_dropoff_datetime', "dropoff_datetime")

In [6]:
df_yellow=spark.read.parquet('../data/part/yellow/*/*')

In [7]:
df_yellow=df_yellow.withColumnRenamed('tpep_pickup_datetime', "pickup_datetime") \
        .withColumnRenamed('tpep_dropoff_datetime', "dropoff_datetime")

In [8]:
set(df_green.columns) and set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [9]:
common_cols =[]
for col in df_green.columns:
    if col in set(df_yellow.columns):
        common_cols.append(col)

In [12]:
df_green_sel=df_green \
    .select(common_cols).withColumn('service_type', F.lit('green'))

In [13]:
df_yellow_sel=df_yellow \
    .select(common_cols).withColumn('service_type', F.lit('yellow'))

In [14]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [15]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [16]:
df_trips_data.registerTempTable('trips_data')

/root/spark/spark-3.3.0-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
spark.sql("""
SELECT service_type,
       count(1)
FROM trips_data 
GROUP BY 
    service_type;
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [18]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 
    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,
    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")


In [19]:
df_result.coalesce(1).write.parquet('../data/report/revenue/', mode='overwrite')

22/11/14 07:52:37 WARN TaskSetManager: Lost task 0.0 in stage 11.0 (TID 65) (10.132.0.2 executor 0): java.io.IOException: Mkdirs failed to create file:/home/josh/de-taxi/batch_processing/data/report/revenue/_temporary/0/_temporary/attempt_20221114075236898968617452491234_0011_m_000000_65 (exists=false, cwd=file:/home/josh/spark/spark-3.3.0-bin-hadoop3/work/app-20221114070201-0000/0)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:329)
	at org.apache.parquet.hadoop.ParquetOutputFormat.getRecordWriter(ParquetOutputFormat.java:482)
	at org.apache.parquet.hadoop.ParquetOutputFormat.getRecordW

Py4JJavaError: An error occurred while calling o126.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org